<a href="https://colab.research.google.com/github/ahmedmabrook/DCGAN/blob/main/DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building GAN



Importing Libraries

In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt


Building Generator

In [14]:
class Generator(nn.Module):
  def __init__(self):
    super(Generator, self).__init__()
    #fully connected layer, expects 100 features as input
    # and it output a 1D vector of size 7*7*128 = 6272
    # also it has a RelU activation
    self.dense = nn.Sequential(
        nn.Linear(100, 7 * 7 * 128),
        nn.ReLU()
    )

    self.conv_layers = nn.Sequential(
        #Block #1 (7x7)
        nn.Upsample(scale_factor=2),
        nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
        nn.BatchNorm2d(128, momentum=0.8),
        nn.ReLU(),

        #Block #2 (14x14)
        nn.Upsample(scale_factor=2),
        nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, padding=1),
        nn.BatchNorm2d(64, momentum=0.8),
        nn.ReLU(),

        #Block #3 (28x28 -> 1)
        nn.Conv2d(in_channels=64, out_channels=1, kernel_size=3, padding=1),
        #using tanh instead of ReLu to avoid generating pixels > +255
        #tanh is normalized between -1 and 1
        nn.Tanh(),
    )

    def forward(self,z):
      x = self.dense(z)
      #reshaping the 1D vector into a 128x7x7
      x = x.view(x.shape[0], 128, 7, 7)
      x = self.conv_layers(x)
      return x



In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [18]:
gen = Generator().to(device)
print(gen)

Generator(
  (dense): Sequential(
    (0): Linear(in_features=100, out_features=6272, bias=True)
    (1): ReLU()
  )
  (conv_layers): Sequential(
    (0): Upsample(scale_factor=2.0, mode='nearest')
    (1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.8, affine=True, track_running_stats=True)
    (3): ReLU()
    (4): Upsample(scale_factor=2.0, mode='nearest')
    (5): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.8, affine=True, track_running_stats=True)
    (7): ReLU()
    (8): Conv2d(64, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): Tanh()
  )
)


Building Discriminator

In [16]:
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator, self).__init__()

    self.conv = nn.Sequential(
      nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1, stride=2),
      nn.LeakyReLU(0.2),
      nn.Dropout(0.25),
      #(14x14 -> 7x7)
      nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1, stride=2),
      #add nothing to left, column of zeros to right
      #add nothing to top, column of zeros to bottom
      #(7x7 -> 8x8)
      nn.ZeroPad2d((0,1,0,1)),
      nn.BatchNorm2d(64, momentum=0.8),
      nn.LeakyReLU(0.2),
      nn.Dropout(0.25),

      nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1, stride=2),
      nn.BatchNorm2d(128, momentum=0.8),
      nn.LeakyReLU(0.2),
      nn.Dropout(0.25),

      nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
      nn.BatchNorm2d(256, momentum=0.8),
      nn.LeakyReLU(0.2),
      nn.Dropout(0.25),

      nn.Flatten(),
      nn.Linear(256 * 4 * 4, 1),
      nn.Sigmoid()
    )


    def forward(self, img):
      x = self.conv(img)
      return x



In [19]:
disc = Discriminator().to(device)
print(disc)

Discriminator(
  (conv): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Dropout(p=0.25, inplace=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): ZeroPad2d((0, 1, 0, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.8, affine=True, track_running_stats=True)
    (6): LeakyReLU(negative_slope=0.2)
    (7): Dropout(p=0.25, inplace=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.8, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2)
    (11): Dropout(p=0.25, inplace=False)
    (12): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.8, affine=True, track_running_stats=True)
    (14): LeakyReLU(negative_slope=0.2)
    (15): Dropout(p=0.25, inplace=False)
    (16): Flatten(start_dim=1, end_di